In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
tr,ts = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv'),pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
tr = tr.select_dtypes(exclude=['object'])
tr.dropna(inplace=True)
tr.head(15)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
5,6,50,85.0,14115,5,5,1993,1995,0.0,732,...,40,30,0,320,0,0,700,10,2009,143000
6,7,20,75.0,10084,8,5,2004,2005,186.0,1369,...,255,57,0,0,0,0,0,8,2007,307000
8,9,50,51.0,6120,7,5,1931,1950,0.0,0,...,90,0,205,0,0,0,0,4,2008,129900
9,10,190,50.0,7420,5,6,1939,1950,0.0,851,...,0,4,0,0,0,0,0,1,2008,118000
10,11,20,70.0,11200,5,5,1965,1965,0.0,906,...,0,0,0,0,0,0,0,2,2008,129500


In [4]:
ts = ts.select_dtypes(exclude=['object'])
ts.dropna(inplace=True)
ts.head(15)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,506.0,0,82,0,0,144,0,0,1,2010
5,1466,60,75.0,10000,6,5,1993,1994,0.0,0.0,...,440.0,157,84,0,0,0,0,0,4,2010
7,1468,60,63.0,8402,6,5,1998,1998,0.0,0.0,...,393.0,0,75,0,0,0,0,0,5,2010
8,1469,20,85.0,10176,7,5,1990,1990,0.0,637.0,...,506.0,192,0,0,0,0,0,0,2,2010
9,1470,20,70.0,8400,4,5,1970,1970,0.0,804.0,...,525.0,240,0,0,0,0,0,0,4,2010
10,1471,120,26.0,5858,7,5,1999,1999,0.0,1051.0,...,511.0,203,68,0,0,0,0,0,6,2010


In [5]:
x,y = tr.drop(['SalePrice'],axis=1,inplace=False),tr['SalePrice']
x_tr,x_ts,y_tr,y_ts = train_test_split(x,y,test_size=0.2,random_state=10)

In [6]:
model1 = LinearRegression()
model1.fit(x_tr,y_tr)
model1.score(x_ts,y_ts)

0.8282267801688638

In [7]:
sqrt(mean_squared_error(y_ts,model1.predict(x_ts)))

37220.750647549154

In [8]:
df = ts[['Id']]
df['SalePrice'] = model1.predict(ts)
df.head(15)

,Id,SalePrice
0,1461,116106.323898
1,1462,112216.117311
2,1463,173367.163814
3,1464,197406.405724
4,1465,200716.348862
5,1466,184000.722866
7,1468,170838.883378
8,1469,205732.136173
9,1470,117077.171408
10,1471,203146.033603


In [9]:
tr,ts = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv'),pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [10]:
mp = ts.dtypes.to_dict()
ohe = LabelEncoder()
for a in mp:
    try:
        if mp[a]=='object':
            ts[a] = ohe.fit_transform(ts[a])
    except:
        ts.drop([a],axis=1,inplace=True)
ts.dropna(inplace=True)
ts.drop(['Electrical'],axis=1,inplace=True)
ts.head(15)

,Id,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1461,20,80.0,11622,1,3,3,4,0,12,...,140,0,0,0,120,0,0,6,2010,4
1,1462,20,81.0,14267,1,0,3,0,0,12,...,393,36,0,0,0,0,12500,6,2010,4
2,1463,60,74.0,13830,1,0,3,4,0,8,...,212,34,0,0,0,0,0,3,2010,4
3,1464,60,78.0,9978,1,0,3,4,0,8,...,360,36,0,0,0,0,0,6,2010,4
4,1465,120,43.0,5005,1,0,1,4,0,22,...,0,82,0,0,144,0,0,1,2010,4
5,1466,60,75.0,10000,1,0,3,0,0,8,...,157,84,0,0,0,0,0,4,2010,4
7,1468,60,63.0,8402,1,0,3,4,0,8,...,0,75,0,0,0,0,0,5,2010,4
8,1469,20,85.0,10176,1,3,3,4,0,8,...,192,0,0,0,0,0,0,2,2010,4
9,1470,20,70.0,8400,1,3,3,0,0,12,...,240,0,0,0,0,0,0,4,2010,4
10,1471,120,26.0,5858,1,0,3,2,0,12,...,203,68,0,0,0,0,0,6,2010,4


In [11]:
tr.drop(['Electrical'],axis=1,inplace=True)
mp = tr.dtypes.to_dict()
ohe = LabelEncoder()
st = set(ts.columns)
st.add('SalePrice')
for a in tr.columns:
    if a not in st:
        tr.drop([a],axis=1,inplace=True)
    elif mp[a]=='object':
        tr[a] = ohe.fit_transform(tr[a])
tr.dropna(inplace=True)
tr.head(15)

,Id,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition,SalePrice
0,1,60,65.0,8450,1,3,3,4,0,5,...,61,0,0,0,0,0,2,2008,4,208500
1,2,20,80.0,9600,1,3,3,2,0,24,...,0,0,0,0,0,0,5,2007,4,181500
2,3,60,68.0,11250,1,0,3,4,0,5,...,42,0,0,0,0,0,9,2008,4,223500
3,4,70,60.0,9550,1,0,3,0,0,6,...,35,272,0,0,0,0,2,2006,0,140000
4,5,60,84.0,14260,1,0,3,2,0,15,...,84,0,0,0,0,0,12,2008,4,250000
5,6,50,85.0,14115,1,0,3,4,0,11,...,30,0,320,0,0,700,10,2009,4,143000
6,7,20,75.0,10084,1,3,3,4,0,21,...,57,0,0,0,0,0,8,2007,4,307000
8,9,50,51.0,6120,1,3,3,4,0,17,...,0,205,0,0,0,0,4,2008,0,129900
9,10,190,50.0,7420,1,3,3,0,0,3,...,4,0,0,0,0,0,1,2008,4,118000
10,11,20,70.0,11200,1,3,3,4,0,19,...,0,0,0,0,0,0,2,2008,4,129500


In [12]:
x,y = tr.drop(['SalePrice'],axis=1,inplace=False),tr['SalePrice']
x_tr,x_ts,y_tr,y_ts = train_test_split(x,y,test_size=0.2,random_state=10)

In [13]:
model2 = LinearRegression()
model2.fit(x_tr,y_tr)
model2.score(x_ts,y_ts)

0.84887771863468